In [3]:
import pandas as pd

df = pd.read_excel('Online Retail.xlsx')
b_matrix = pd.crosstab(df['InvoiceNo'], df['StockCode'])
#print(b_matrix)
b_matrix.to_excel('binary_matrix.xlsx')
b_matrix = (b_matrix > 0).astype(int)

#print(b_matrix)
print("hello")

KeyboardInterrupt: 

step 3

In [1]:
import pandas as pd

df = pd.read_excel('Online Retail.xlsx')
df_sorted = df.sort_values('InvoiceDate')
batch_size = 10000
batches = [df_sorted.iloc[i:i + batch_size] for i in range(0, len(df_sorted), batch_size)]




     InvoiceNo StockCode                          Description  Quantity  \
0       536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1       536365     71053                  WHITE METAL LANTERN         6   
2       536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3       536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4       536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...        ...       ...                                  ...       ...   
9996    537224     22623        BOX OF VINTAGE JIGSAW BLOCKS          2   
9917    537224     22866        HAND WARMER SCOTTY DOG DESIGN         1   
9994    537224     21361             LOVE LARGE WOOD LETTERS          1   
9932    537224    84531A                PINK KNITTED EGG COSY         1   
9933    537224     22087             PAPER BUNTING WHITE LACE         1   

             InvoiceDate  UnitPrice  CustomerID         Country  
0    2010-12-01 08:26:00       2.

step 4

In [1]:
import pandas as pd
import numpy as np
from sklearn.random_projection import GaussianRandomProjection
from sklearn.decomposition import IncrementalPCA

df = pd.read_excel('Online Retail.xlsx')
df_sorted = df.sort_values('InvoiceDate')
batch_size = 10000
batches = [df_sorted.iloc[i:i + batch_size] for i in range(0, len(df_sorted), batch_size)]

x = 1
for batch in batches:
  #print(batch)
  binary_matrix = pd.crosstab(batch['InvoiceNo'], batch['StockCode'])
  binary_matrix = (binary_matrix > 0).astype(int)
  X = binary_matrix.to_numpy()


  rgp = GaussianRandomProjection(n_components=50)
  rgp_result = rgp.fit_transform(X)
  print(f"Random Gaussian Projection result shape: {rgp_result.shape}")

  ipca = IncrementalPCA(n_components=50)
  ipca.partial_fit(X)
  ipca_result = ipca.transform(X)
  print(f"Incremental PCA result shape: {ipca_result.shape}")


  #print(binary_matrix)
  if x:
    exit()


Random Gaussian Projection result shape: (512, 50)
Incremental PCA result shape: (512, 50)
Random Gaussian Projection result shape: (427, 50)
Incremental PCA result shape: (427, 50)
Random Gaussian Projection result shape: (468, 50)
Incremental PCA result shape: (468, 50)
Random Gaussian Projection result shape: (519, 50)
Incremental PCA result shape: (519, 50)
Random Gaussian Projection result shape: (415, 50)
Incremental PCA result shape: (415, 50)
Random Gaussian Projection result shape: (402, 50)
Incremental PCA result shape: (402, 50)
Random Gaussian Projection result shape: (315, 50)
Incremental PCA result shape: (315, 50)
Random Gaussian Projection result shape: (578, 50)
Incremental PCA result shape: (578, 50)
Random Gaussian Projection result shape: (555, 50)
Incremental PCA result shape: (555, 50)
Random Gaussian Projection result shape: (473, 50)
Incremental PCA result shape: (473, 50)
Random Gaussian Projection result shape: (512, 50)
Incremental PCA result shape: (512, 50)